In [1]:
import os
import sys
home_dir = "../"
module_path = os.path.abspath(os.path.join(home_dir))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import pandas as pd

import utility.pickle_utils as pickle_utils
from analysis_motif_finding.motif_utils import find_high_attention, merge_motifs, make_window#, motifs_hypergeom_test, filter_motifs, 

In [3]:
# args
min_len=10
min_times_cons=20
return_idx=False
pval_cutoff=0.005
align_all_ties=True
window_size=15
min_n_motif=10

settings_str = f"settings-min_len={min_len}-min_times_cons={pval_cutoff:.3f}-min_len={min_len}-min_n_motif={min_n_motif}-window_size={window_size}"
print(settings_str)
high_attn_regions_settings_str = f"min_len={min_len}-min_times_cons={min_times_cons}"
high_attn_regions_settings_str

settings-min_len=10-min_times_cons=0.005-min_len=10-min_n_motif=10-window_size=15


'min_len=10-min_times_cons=20'

In [4]:
from utility.dnabert2 import get_dnabert2_tokenizer

# loading things
tokenizer = get_dnabert2_tokenizer(max_num_tokens=512, home_dir=home_dir)
# print(tokenizer)

avg_crossattn_weights = pickle_utils.load(home_dir+"analysis/weights/all_test_seq_avg_crossattn_weights_list.pkl")
print(len(avg_crossattn_weights))

data_df = pd.read_csv(home_dir+"data/train_val_test/peaks_with_labels_test.tsv.gz", compression="gzip", sep="\t")
print(data_df.shape)

seq_dict = pickle_utils.load(home_dir+"data/processed/seq_with_flanks_dict.pkl") # all seqs
print(len(seq_dict))

neg_seq_dict = pickle_utils.load(home_dir+"data/processed/seq_neg_with_flanks_dict.pkl")
neg_seqs = [v for k, v in neg_seq_dict.items()]
len(neg_seqs), len(neg_seqs[0])

/usr/projects/pyDNA_EPBD/tf_dna_binding/.venvs/python311_conda_3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


165321
(165321, 5)
1903669


(165321, 1000)

In [4]:
# dev = pd.read_csv(home_dir+"analysis/dnabert_motif_analysis_data/dev.tsv",sep='\t',header=0)
# dev.columns = ['sequence','label']
# dev['seq'] = dev['sequence'].apply(kmer2seq)
# dev_pos = dev[dev['label'] == 1].reset_index(drop=True)
# dev_neg = dev[dev['label'] == 0].reset_index(drop=True)
# neg_seqs = dev_neg['seq']
# pos_seqs = dev_pos['seq']
# len(neg_seqs[0]), len(pos_seqs[0])
# dev_pos

In [5]:
# def get_epbd_features(fname):
#     fpath = home_dir+"data/pydnaepbd_things/features/id_seqs/"+fname
#     data = pickle_utils.load(fpath)
    
#     # coord and flip features
#     coord = np.expand_dims(data["coord"], axis=0)
#     flips = np.transpose(data["flip_verbose"])
#     epbd_features = np.concatenate([coord, flips], axis=0) / 80000
#     epbd_features = torch.tensor(epbd_features, dtype=torch.float32)
#     # print(epbd_features.shape) # [6, 200]
#     return epbd_features

In [5]:
motif_seqs = {}
pos_seqs = []
for i in range(data_df.shape[0]):
    # getting seq
    x = data_df.loc[i]
    chrom, start, end = x["chrom"], int(x["start"]), int(x["end"])
    seq_id = f"{chrom}_{str(start)}_{str(end)}"
    seq = seq_dict[seq_id]
    # print(len(seq))

    # pos_seq = seq[400:400+200] # need to keep the whole since they are tokenized
    pos_seqs.append(seq)

    toked = tokenizer(
                seq,
                return_tensors="pt",
                padding="longest",
                max_length=512,
                truncation=True,
            )
    # print(toked["input_ids"][0])
    toked_seq_list= tokenizer.decode(toked["input_ids"][0]).split()
    toked_seq_list = toked_seq_list[1:-1] # removing cls and sep token
    # print(toked_seq_list)
    # print(len(toked_seq_list))

    # cross-attn-weights of the seq from model
    cross_attn_w = avg_crossattn_weights[i]
    cross_attn_w = cross_attn_w[1:-1]
    # print(cross_attn_w.shape)

    expanded_cross_attn_w = []
    for j, w in enumerate(cross_attn_w):
        seq_token_len = len(toked_seq_list[j])
        expanded_cross_attn_w += [w]*seq_token_len
    expanded_cross_attn_w = np.array(expanded_cross_attn_w)    
    # print(expanded_cross_attn_w.shape)
    # print(expanded_cross_attn_w)

    # epbd_features = get_epbd_features(f"{seq_id}.pkl")
    # print(cross_attn_w.shape, epbd_features.shape)
    # flip = np.array(epbd_features[5])

    # finding motif regions
    motif_regions = find_high_attention(expanded_cross_attn_w, min_len=min_len, min_times_cons=min_times_cons)
    # motif_regions = find_high_attention(flip, min_len=min_len)
    # print(motif_regions)

    # print(motif_idx[0], motif_idx[1])
    # print(cross_attn_w[motif_idx[0]:motif_idx[1]+1]) # do not need to include this 1
    # plt.plot(cross_attn_w)

    for motif_idx in motif_regions:
        # seq = toked_seq_list[motif_idx[0]:motif_idx[1]]
        # seq = "".join(seq)
        # # print(seq, len(seq))

        # start_pos_in_real_seq = len("".join(toked_seq_list[1:motif_idx[0]]))
        # end_pos_in_real_seq = len("".join(toked_seq_list[1:motif_idx[1]]))
        # print(start_pos_in_real_seq, end_pos_in_real_seq, end_pos_in_real_seq-start_pos_in_real_seq, len(seq))

        
        start_pos_in_real_seq = motif_idx[0]
        end_pos_in_real_seq = motif_idx[1]
        seq = seq[start_pos_in_real_seq:end_pos_in_real_seq]

        if motif_idx[0] >= motif_idx[1]:
            raise Exception(f"Starting motif index ({motif_idx[0]}) can never be ending motif index ({motif_idx[1]})")
        
        if seq not in motif_seqs:
            motif_seqs[seq] = {'seq_idx': [i], 'atten_region_pos':[(start_pos_in_real_seq, end_pos_in_real_seq)]}
        else:
            motif_seqs[seq]['seq_idx'].append(i)
            motif_seqs[seq]['atten_region_pos'].append((start_pos_in_real_seq, end_pos_in_real_seq))

    
    if i%10000==0: print(i, len(motif_seqs))
    # if i==10: break 
    # break

0 2
10000 5238
20000 10567
30000 15924
40000 21425
50000 27166
60000 32661
70000 38426
80000 43698
90000 48060
100000 53549
110000 58740
120000 64194
130000 68976
140000 74373
150000 79330
160000 83574


In [6]:
print(len(pos_seqs), len(neg_seqs))
print(len(pos_seqs[0]), len(neg_seqs[0]))
print(len(motif_seqs))
# motif_seqs

165321 165321
1000 1000
85463


In [7]:
def count_motif_instances(seqs, motifs, allow_multi_match=False):
    """
    Use Aho-Corasick algorithm for efficient multi-pattern matching
    between input sequences and motif patterns to obtain counts of instances.

    Arguments:
    seqs -- list, numpy array or pandas series of DNA sequences
    motifs -- list, numpy array or pandas series, a collection of motif patterns
        to be matched to seqs

    Keyword arguments:
    allow_multi_match -- bool, whether to allow for counting multiple matchs (default False)

    Returns:
    motif_count -- count of motif instances (int)

    """
    import ahocorasick
    from operator import itemgetter

    motif_count = {}

    A = ahocorasick.Automaton()
    for idx, key in enumerate(motifs):
        A.add_word(key, (idx, key))
        motif_count[key] = 0
    A.make_automaton()

    for i, seq in enumerate(seqs):
        matches = sorted(map(itemgetter(1), A.iter(seq)))
        matched_seqs = []
        for match in matches:
            match_seq = match[1]
            assert match_seq in motifs
            if allow_multi_match:
                motif_count[match_seq] += 1
            else:  # for a particular seq, count only once if multiple matches were found
                if match_seq not in matched_seqs:
                    motif_count[match_seq] += 1
                    matched_seqs.append(match_seq)
        if i%1000==0:
            print(i)

    return motif_count


def motifs_hypergeom_test(
    pos_seqs,
    neg_seqs,
    motifs,
    p_adjust="fdr_bh",
    alpha=0.05,
    verbose=False,
    allow_multi_match=False,
    **kwargs
):
    """
    Perform hypergeometric test to find significantly enriched motifs in positive sequences.
    Returns a list of adjusted p-values.

    Arguments:
    pos_seqs -- list, numpy array or pandas series of positive DNA sequences
    neg_seqs -- list, numpy array or pandas series of negative DNA sequences
    motifs -- list, numpy array or pandas series, a collection of motif patterns
        to be matched to seqs

    Keyword arguments:
    p_adjust -- method used to correct for multiple testing problem. Options are same as
        statsmodels.stats.multitest (default 'fdr_bh')
    alpha -- cutoff FDR/p-value to declare statistical significance (default 0.05)
    verbose -- verbosity argument (default False)
    allow_multi_match -- bool, whether to allow for counting multiple matchs (default False)

    Returns:
    pvals -- a list of p-values.

    """
    from scipy.stats import hypergeom
    import statsmodels.stats.multitest as multi

    pvals = []
    N = len(pos_seqs) + len(neg_seqs)
    K = len(pos_seqs)
    print("counting motif seqs in all seqs")
    motif_count_all = count_motif_instances(
        pos_seqs + neg_seqs, motifs, allow_multi_match=allow_multi_match
    )
    print("counting motif seqs in pos seqs")
    motif_count_pos = count_motif_instances(
        pos_seqs, motifs, allow_multi_match=allow_multi_match
    )

    for i, motif in enumerate(motifs):
        print(i)
        n = motif_count_all[motif]
        x = motif_count_pos[motif]
        pval = hypergeom.sf(x - 1, N, K, n)
        if verbose:
            if pval < 1e-5:
                print(
                    "motif {}: N={}; K={}; n={}; x={}; p={}".format(
                        motif, N, K, n, x, pval
                    )
                )
        #         pvals[motif] = pval
        # if i % 1000 == 0:
        
        pvals.append(pval)

    # adjust p-value
    if p_adjust is not None:
        pvals = list(multi.multipletests(pvals, alpha=alpha, method=p_adjust)[1])
    return pvals

In [8]:
pvals_saved_file = home_dir+f"analysis_motif_finding/p_values/{high_attn_regions_settings_str}.pkl"
if os.path.exists(pvals_saved_file):
    pvals = pickle_utils.load(pvals_saved_file)
else:
    pvals = motifs_hypergeom_test(pos_seqs, neg_seqs, list(motif_seqs.keys()))
    pickle_utils.save(pvals, pvals_saved_file)
len(pvals)

85463

In [9]:
# pvals statistics
print(np.min(pvals), np.max(pvals), np.median(pvals))

1.1596161006981353e-153 1.0 0.22820597434628076


In [10]:
# motifs_to_keep = filter_motifs(pos_seqs, neg_seqs, list(motif_seqs.keys()), cutoff=np.median(pvals), return_idx=return_idx)
motifs = list(motif_seqs.keys())
pval_cutoff = 0.005#np.median(pvals)
motifs_to_keep = [motifs[i] for i, pval in enumerate(pvals) if pval <= pval_cutoff]
kept_motif_seqs = {k: motif_seqs[k] for k in motifs_to_keep}
print(len(kept_motif_seqs))
kept_motif_seqs_copy = kept_motif_seqs.copy()

5645


In [11]:
merged_motif_seqs = merge_motifs(kept_motif_seqs_copy, min_len=min_len, align_all_ties=True)
print(len(merged_motif_seqs))
# merged_motif_seqs

2375


In [12]:
new_merged_motif_seqs = make_window(merged_motif_seqs, pos_seqs, window_size=window_size) # not correct
print(len(new_merged_motif_seqs))
new_merged_motif_seqs

2375


{'CATTTGCATA': {'seq_idx': [75, 134251, 140044],
  'atten_region_pos': [(548, 563), (448, 463), (414, 429)],
  'seqs': ['TTCATTTGCATACGG', 'AGCATGTGCATATCC', 'CCCATTTGCATACAA']},
 'GTCATTCCTG': {'seq_idx': [83, 54447, 63340, 2776, 26572, 40811],
  'atten_region_pos': [(537, 552),
   (461, 476),
   (594, 609),
   (555, 570),
   (463, 478),
   (507, 522)],
  'seqs': ['CAGTCATTCCTGGAG',
   'GAGTCATTCCTGTCC',
   'GAGTCATTCCTGGAA',
   'CCGTGATTCCTGGCA',
   'CAGTGATTCCTGTCC',
   'CAGTCATTTCTGGGA']},
 'CAGGAATGTT': {'seq_idx': [103,
   11499,
   595,
   596,
   23112,
   38928,
   38929,
   43751,
   45801,
   55896,
   112987,
   113726,
   149612,
   163332,
   58608,
   126959,
   45586,
   70767],
  'atten_region_pos': [(429, 444),
   (486, 501),
   (589, 604),
   (389, 404),
   (434, 449),
   (523, 538),
   (323, 338),
   (416, 431),
   (529, 544),
   (535, 550),
   (537, 552),
   (339, 354),
   (520, 535),
   (549, 564),
   (585, 600),
   (364, 379),
   (386, 401),
   (475, 490)],
  'se

In [13]:
new_new_merged_motif_seqs = {k: coords for k, coords in new_merged_motif_seqs.items() if len(coords['seq_idx']) >= min_n_motif}
print(len(new_new_merged_motif_seqs))
new_new_merged_motif_seqs

124


{'CAGGAATGTT': {'seq_idx': [103,
   11499,
   595,
   596,
   23112,
   38928,
   38929,
   43751,
   45801,
   55896,
   112987,
   113726,
   149612,
   163332,
   58608,
   126959,
   45586,
   70767],
  'atten_region_pos': [(429, 444),
   (486, 501),
   (589, 604),
   (389, 404),
   (434, 449),
   (523, 538),
   (323, 338),
   (416, 431),
   (529, 544),
   (535, 550),
   (537, 552),
   (339, 354),
   (520, 535),
   (549, 564),
   (585, 600),
   (364, 379),
   (386, 401),
   (475, 490)],
  'seqs': ['TGCAGGAATGTTGTG',
   'TTCAGGAATGTTCCG',
   'CACAGGAATGTGCCT',
   'CACAGGAATGTGCCT',
   'TTCAGGAATGTGCCC',
   'GACAGGAATGTGGTT',
   'GACAGGAATGTGGTT',
   'TTCAGGAATGTGCAC',
   'GGCAGGAATGTGTTA',
   'TCCAGGAATGTGGAC',
   'CCCAGGAATGTGCTC',
   'TACAGGAATGTGCAT',
   'GACAGGAATGTGCCT',
   'CCCAGGAATGTGGGT',
   'GGCAGGAATGTGATA',
   'GCCAGGAATGTGTGA',
   'TCCAGGAATGTTCAG',
   'GACTGGAATGTTCTT']},
 'TTATGAGTCA': {'seq_idx': [128,
   129,
   14741,
   34762,
   60591,
   7340,
   129476,
   1737

In [14]:
from Bio import motifs
from Bio.Seq import Seq

save_file_dir = home_dir+f"analysis_motif_finding/{settings_str}/"
if save_file_dir is not None:
    os.makedirs(save_file_dir, exist_ok=True)
    for motif_no, (motif, instances) in enumerate(new_new_merged_motif_seqs.items()):
        # saving to files
        with open(save_file_dir+'/motif_{}_{}_{}.txt'.format(motif_no+1, motif, len(instances['seq_idx'])), 'w') as f:
            for seq in instances['seqs']:
                f.write(seq+'\n')
        # make weblogo
        seqs = [Seq(v) for i,v in enumerate(instances['seqs'])]
        # print(seqs)
        m = motifs.create(seqs)
        # m.weblogo(save_file_dir+"/motif_{}_{}_weblogo.png".format(motif, len(instances['seq_idx'])), format='png_print',
        #                     show_fineprint=False, show_ends=False, color_scheme='color_classic')